# Data Migration Tracking - Optimized

### Description
To track data migration status and filings done after migration - Optimized version with fewer queries and dataframes

#### Install Dependencies

In [ ]:
# Run this cell if you haven't installed these packages
!pip install pandas openpyxl sqlalchemy numpy psycopg2-binary python-dotenv


### Setup and COnfiguration

In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from typing import List, Any
from openpyxl.styles import Font, Alignment
from datetime import datetime

load_dotenv()

# Configuration
GROUP_TABLE_FOLDER = os.getenv('GROUP_TABLE_FOLDER')
GROUP_TABLE_FILE_NAME = os.getenv('GROUP_TABLE_FILE_NAME')
OUTPUT_FOLDER = os.getenv('OUTPUT_FOLDER')
COLUMN_FOR_CORP_NUM = os.getenv('COLUMN_FOR_CORP_NUM')

# Colin Extracts DB
COLIN_EXTRACT_DB = os.getenv('COLIN_EXTRACT_DB')
CE_HOST_URL = os.getenv('CE_HOST_URL')
CE_USERNAME = os.getenv('CE_USERNAME')
CE_PASSWORD = os.getenv('CE_PASSWORD')
CE_PORT = os.getenv('CE_PORT')

# Lear DB
LEAR_DB = os.getenv('LEAR_DB')
LEAR_HOST_URL = os.getenv('LEAR_HOST_URL')
LEAR_USERNAME = os.getenv('LEAR_USERNAME')
LEAR_PASSWORD = os.getenv('LEAR_PASSWORD')
LEAR_PORT = os.getenv('LEAR_PORT')

# Column mapping
COLUMN_NAMES = {
    "corp_num": "Incorporation Number",
    "corp_name": "Company Name",
    "corp_type": "Type",
    "email": "Admin Email",
    "status": "Migration Status",
    "date": "Migrated Date",
    "filings": "Filings Done",
    "filing_date": "Last Filing Date"
}

PRINT_DIVIDER = "=" * 50


In [ ]:
def get_db_connection_string(host_address: str, database: str, user_name: str, db_password: str, port: str = "5432") -> str:
    """Create db connection string."""
    return f"postgresql://{user_name}:{db_password}@{host_address}:{port}/{database}"


### Read Excel file and extract corp_nums

In [ ]:
try:
    full_group_table_path = f"{GROUP_TABLE_FOLDER}/{GROUP_TABLE_FILE_NAME}"
    corp_nums_df = pd.read_excel(full_group_table_path, sheet_name="Sheet1", usecols=[COLUMN_FOR_CORP_NUM])
    corp_nums_df = corp_nums_df.sort_values(COLUMN_FOR_CORP_NUM)
    
    corp_num_column_values = corp_nums_df[COLUMN_FOR_CORP_NUM].dropna().tolist()
    
    print(f"Found {len(corp_num_column_values)} corps in the group table")
    print(f"All corps: {corp_num_column_values}")
    print(PRINT_DIVIDER)

except Exception as e:
    print(f"Error reading Excel file: {e}")
    corp_num_column_values = []


### Connect to Colin Extracts db and get all Data

In [ ]:
try:
    colin_extracts_connection_string = get_db_connection_string(
        CE_HOST_URL, COLIN_EXTRACT_DB, CE_USERNAME, CE_PASSWORD, CE_PORT
    )
    colin_extracts_engine = create_engine(colin_extracts_connection_string)
    
    # Test connection
    with colin_extracts_engine.connect() as conn:
        conn.execute(text("SELECT 1"))
    print("Colin Extracts database connection successful")

except Exception as e:
    print(f"Connection to Colin Extracts failed: {e}")
    colin_extracts_engine = None

print(PRINT_DIVIDER)


In [ ]:
# Get corp_name, corp_type, admin_email, migration_status, migration_date from Colin Extracts
if colin_extracts_engine and corp_num_column_values:
    try:
        values_str = "', '".join(str(val) for val in corp_num_column_values)
        in_clause = f"corp_num IN ('{values_str}')"

        # Single comprehensive query joining all needed tables
        combined_query = f"""
        SELECT 
            corp.corp_num as "{COLUMN_NAMES['corp_num']}",
            cn.corp_name as "{COLUMN_NAMES['corp_name']}",
            corp.corp_type_cd as "{COLUMN_NAMES['corp_type']}",
            corp.admin_email as "{COLUMN_NAMES['email']}",
            CASE 
                WHEN cp.processed_status = 'COMPLETED' THEN 'Migrated'
                ELSE 'Pending'
            END as "{COLUMN_NAMES['status']}",
            cp.create_date::date as "{COLUMN_NAMES['date']}"
        FROM public.corporation corp
        LEFT JOIN public.corp_name cn ON corp.corp_num = cn.corp_num 
            AND cn.corp_name_typ_cd IN ('CO', 'NB') 
            AND cn.end_event_id IS NULL
        LEFT JOIN public.corp_processing cp ON corp.corp_num = cp.corp_num
        WHERE corp.{in_clause}
        ORDER BY corp.corp_num
        """
        
        # Execute combined query
        tracking_df = pd.read_sql(combined_query, colin_extracts_engine)
        
        # Convert date column
        tracking_df[COLUMN_NAMES['date']] = pd.to_datetime(tracking_df[COLUMN_NAMES['date']])
        
        print(f"Retrieved data for {len(tracking_df)} corporations from Colin Extracts")
        display(tracking_df.head())
        print(PRINT_DIVIDER)
        
    except Exception as e:
        print(f"Failed to execute combined query: {e}")
        tracking_df = pd.DataFrame()


### Connect to LEAR DB and get post-migration filings data

In [ ]:
# Connect to LEAR DB
try:
    lear_connection_string = get_db_connection_string(
        LEAR_HOST_URL, LEAR_DB, LEAR_USERNAME, LEAR_PASSWORD, LEAR_PORT
    )
    lear_engine = create_engine(lear_connection_string)
    
    # Test connection
    with lear_engine.connect() as lear_conn:
        lear_conn.execute(text("SELECT 1"))
    print("LEAR database connection successful")

except Exception as e:
    print(f"Connection to LEAR failed: {e}")
    lear_engine = None

print(PRINT_DIVIDER)


In [ ]:
# Get post-migration filings data (filing types and last filing date) from LEAR
def convert_filings_to_title_case(filings_string: str) -> str:
        """Convert camelCase to Title Case (e.g., 'annualReport' -> 'Annual Report')"""
        import re
        result = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', filings_string)
        return result.title()


if lear_engine and corp_num_column_values:
    try:
        values_str = "', '".join(str(val) for val in corp_num_column_values)
        in_clause = f"b.identifier IN ('{values_str}')"
        
        # Combined query to get business IDs and filings in one go
        lear_combined_query = f"""
        WITH filing_summary AS (
            SELECT 
                b.identifier,
                STRING_AGG(
                    DISTINCT filing_type,
                    ', '
                ) as filings_done,
                MAX(f.filing_date)::date as last_filing_date
            FROM public.businesses b
            LEFT JOIN public.filings f ON b.id = f.business_id 
            AND f.source = 'LEAR'
            AND f.status = 'COMPLETED'
            WHERE {in_clause}
            GROUP BY b.identifier
        )
        SELECT 
            identifier as "{COLUMN_NAMES['corp_num']}",
            COALESCE(filings_done, '') as "{COLUMN_NAMES['filings']}",
            COALESCE(last_filing_date::text, '') as "{COLUMN_NAMES['filing_date']}"
        FROM filing_summary
        ORDER BY identifier
        """
        
        lear_filings_df = pd.read_sql(lear_combined_query, lear_engine)
        
        print(f"Retrieved LEAR filings data for {len(lear_filings_df)} corporations")
        if not lear_filings_df.empty:
            lear_filings_df[COLUMN_NAMES['filings']] = lear_filings_df[COLUMN_NAMES['filings']].apply(convert_filings_to_title_case)
            display(lear_filings_df)

    except Exception as e:
        print(f"Error getting LEAR filings data: {e}")
        lear_filings_df = pd.DataFrame()
    
print(PRINT_DIVIDER)


### Combine All Data and Save Final Result

In [ ]:
def generate_unique_filename(original_path: str) -> str:
    """Generate unique filename with date and incremental number if file exists."""
    directory = os.path.dirname(original_path)
    filename = os.path.basename(original_path)
    name, ext = os.path.splitext(filename)
    
    today = datetime.now().strftime('%Y%m%d')
    new_filename = f"{name}_{today}{ext}"
    new_path = os.path.join(directory, new_filename)
    
    if not os.path.exists(new_path):
        return new_path
    
    counter = 2
    while counter <= 999:
        incremental_filename = f"{name}_{today}_{counter:02d}{ext}"
        incremental_path = os.path.join(directory, incremental_filename)
        
        if not os.path.exists(incremental_path):
            return incremental_path
        
        counter += 1
    
    raise Exception("Too many files with the same name pattern")


def format_and_save_excel(df: pd.DataFrame, file_save_path: str = 'output.xlsx', sheet_name: str = 'Sheet1') -> None:
    """Save to Excel file with basic freeze and alignment."""
    file_save_path = generate_unique_filename(file_save_path)

    with pd.ExcelWriter(file_save_path, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        worksheet = writer.sheets[sheet_name]
        
        # Freeze the 1st row
        worksheet.freeze_panes = 'A2'
        
        # Make header bold and left-align all cells
        for cell in worksheet[1]:
            cell.font = Font(bold=True)
            cell.alignment = Alignment(horizontal='left')
        
        for row in worksheet.iter_rows(min_row=2):
            for cell in row:
                cell.alignment = Alignment(horizontal='left')
        
        # Adjust column widths
        for column in worksheet.columns:
            max_length = max(len(str(cell.value)) for cell in column if cell.value)
            worksheet.column_dimensions[column[0].column_letter].width = max_length + 2
    
    print(f"DataFrame saved to {file_save_path} with frozen header and left alignment")


In [ ]:
try:
    if not tracking_df.empty:
        # Merge with LEAR filings data
        if not lear_filings_df.empty:
            final_df = tracking_df.merge(
                lear_filings_df,
                on=COLUMN_NAMES['corp_num'],
                how='left'
            )
        else:
            # If no LEAR data, add empty columns
            final_df = tracking_df.copy()
            final_df[COLUMN_NAMES['filings']] = ''
            final_df[COLUMN_NAMES['filing_date']] = ''
        
        # Convert date column to string for Excel export
        final_df[COLUMN_NAMES['date']] = final_df[COLUMN_NAMES['date']].astype(str)
        
        print(f"Final dataset contains {len(final_df)} corporations")
        print(f"Corporations with filings: {len(final_df[final_df[COLUMN_NAMES['filings']] != ''])}")
        
        # Display sample of final data
        with pd.option_context('display.max_rows', None):  # Adjust None to a number when dataset is big
            display(final_df)
        
        # Save to Excel
        output_path = f"{OUTPUT_FOLDER}/migration_tracking_result.xlsx"
        format_and_save_excel(final_df, output_path)
        
        print(PRINT_DIVIDER)
        print("Processing completed successfully!")
        
    else:
        print("No data retrieved from Colin Extracts database")
        
except Exception as e:
    print(f"Error in final data processing: {e}")
